## Vision-Python Demonstration Workbook: Vision Language

*Objective*: To illustrate access to a vision service that delivers standard python-ready json structures with the longer-term goal of demonstrating access to client FAST environments.


### Audience

This notebook contains some basic Vision coding examples that illustrate access and manipulation of data in Vision from your python environment.  Vision expressions can return text or python dictionary structures.  The audience for these examples is expected to have basic familiarity with the Vision language and with python

#### Notes

* This notebook provides a number of examples that fetch data from a Vision federated database utilizing an open-source node-js package talking to vision.  This is a work-in-progress designed to demonstrate concepts.  Only FactSet internals will have access to this database.  All of the components demonstrated here are designed to work with client Vision environments as well.


### Getting Started

To begin, you will need to import the Vision Connection (vconnect) module.  __[Get the <i>vconnect</i> module here](https://github.com/LeslieNewman/Vi-Py/blob/main/py/vconnect.py)__ .  Store this file locally in your normal work area.  The examples will be using several functions from this module:

* *vc.runvget* is used to submit a Vision expression as a *get* url.
* *vc.runvision* is used to run a Vision query and return the results as text
* *vc.runvisionJSON* is used to run a Vision query and return the results as a JSON dictionary.  

The examples below also use the *DataFrame* class from the *pandas* module, so you probably want to import that as well.



In [1]:
import pandas as pd
import vconnect as vc

### Waking up the Vision Service

This demo is leveraging a non-production service running on a dev box.  At any time, the service may be in one of the following states:

1. Available
1. Needing Restart
1. Dead

The following code can be run to determine which state the service is in:

>`
  vc.runvision ('2 + 2')
>`

If the service is **available**, you will see an appropriate response.  If the service is **needing restart**, you will get an error message that includes the text *RemoteDisconnected* after a minute or so.  If you see this message, you should run the code again which should restart the service for you.  This step may take a minute or two to finish.  If you get another error message, the service is most like **dead** and will need manual intervention on the unix account.


In [2]:
#-- wake up the Vision service
print (vc.runvision('2 + 2'))

     4.00



## Running Basic Vision Expressions

The function *vc.runvision* is used to execute a Vision expression supplied as a string parameter, returning text.


In [3]:
#-- As basic as you can get
vc.runvision('2+2')

'     4.00\n'

In [4]:
#-- show all the currencies
vc.runvision('Currency masterListCodes')

'Code   Name\nUSD    US Dollar\nCAD    Canadian Dollar\nEUR    Euro Dollar\nLEGACY LEGACY\nATS    Austrian Schilling\nBEF    Belgian Franc\nFIM    Finnish Markka\nFRF    French Franc\nDEM    German Deutschmark\nIEP    Irish Punt\nITL    Italian Lira\nNLG    Netherlands Guilder\nPTE    Portugese Escudo\nESP    Spanish Peseta\nLUF    Luxembourg Franc\nGRD    GRD\n\n'

In [5]:
#--  make that look prettier
print(vc.runvision('Currency masterListCodes'))

Code   Name
USD    US Dollar
CAD    Canadian Dollar
EUR    Euro Dollar
LEGACY LEGACY
ATS    Austrian Schilling
BEF    Belgian Franc
FIM    Finnish Markka
FRF    French Franc
DEM    German Deutschmark
IEP    Irish Punt
ITL    Italian Lira
NLG    Netherlands Guilder
PTE    Portugese Escudo
ESP    Spanish Peseta
LUF    Luxembourg Franc
GRD    GRD




In [6]:
#--  Federation is available
result = vc.runvision('SecurityMap findId: "FDS" . profile ')
print (result)

SecurityMap entry 303075105
Aliases: CUSIP_303075105 | MSCITSCODE_TS66137 | PERMID_DJC0PL-S-US | SEDOL_2329770 | TICKERUS_FDS | 

 * FactSet   303075105    FDS          FACTSET RESEARCH SYSTEMS INC
   FundPricesDefault      Default      Default
   CSIStandar30307510     FDS          FACTSET RESEARCH SYSTEMS INC
   CSIResearcDefault      Default      Default
   Msci      TS66137      TS66137      FACTSET RESEARCH SYSTEMS
   Worldscope303075105   C    FDS        303075105  2329770   US3030751057  FACTSET RESEARCH SYSTEMS INC.
   IbesDomMM FD1          FD1          FACTSET RESEARCH SYSTEMS INC
   IbesIntMM Default      Default      Default
   SandP     30307510     FDS          FactSet Research System Inc
   Russell   30307510     FDS          FACTSET RESEARCH SYSTEMS
   Topix     Default      Default      Default
   Ftse      C69897.F     C69897.F     Factset Research Systems
   DJStoxx   SC000FDS     SC000FDS     FACTSET RESEARCH SYS.
   LocalVisioDefault      Default      Default




In [7]:
#--  Since we are just returninig text, any Selector Not Founds will flow right through
result = vc.runvision('Currency makeBelieveMessage')
print (result)


>>> Selector 'makeBelieveMessage' Not Found <<<
      NA 



## Running Vision Expressions that Return JSON

The function *vc.runvisionJSON* is used to execute a Vision expression that is designed to return a dictionary object.  There are several ways to use Vision to create an object that can be transported to python via a JSON represenstation.  The following methods have been defined at the new Vision class _JS_ to return a python *array* of *dict* objects:

* *getArrayFrom: block for: itemString*
* *getArrayFrom: block for: itemString over: dateList*

where:
* *block* is a Vision block containing the code to run.  This code shoud return a Vision Collection object
* *itemString* is a string containing a comma-separated set of vision expressions to run for each element of the collection
* *dateList* is a list of dates or a Vision DateRange.  If omitted, ^date is used

These methods return a dictionary structure which includes:

* *rowCount*  :  number of rows returned in the array
* *colCount*  :  number of columns returned in the array
* *items*  :  array of item expressions requested  (the keys)
* *dates*  :  array of dates requests
* *array*  : array of dict objects containing the values for each key
              
   

In [8]:
#--  First example using getArray
#--  List all the currencies and their latest exchange rate

vcode = '''
JS getArrayFrom: [ CurrencyMap factset masterList first: 5 ]
   for: "name,usExchange"
'''
vc.runvisionJSON(vcode)

{'processId': 7179,
 'rowCount': 5,
 'colCount': 4,
 'items': ['id', 'date', 'name', 'usExchange'],
 'dates': [20201130],
 'array': [{'id': 'USD',
   'date': 20201130,
   'usExchange': 1,
   'name': 'U.S. Dollar'},
  {'id': 'CAD',
   'date': 20201130,
   'usExchange': 1.29755,
   'name': 'Canadian Dollar'},
  {'id': 'EUR',
   'date': 20201130,
   'usExchange': 0.83692515,
   'name': 'Euro Dollar'},
  {'id': 'LEGACY',
   'date': 20201130,
   'usExchange': 'visionNA',
   'name': 'LEGACY'},
  {'id': 'ATS',
   'date': 20201130,
   'usExchange': 11.516341141545,
   'name': 'Austrian Schilling'}]}

In [9]:
#-- Let's pretty it up using pandas DataFrame

vcode = '''
JS getArrayFrom: [ CurrencyMap factset masterList first: 10 ]
   for: "name,usExchange"
'''
result = vc.runvisionJSON(vcode)
pd.DataFrame(result['array'])

,date,id,name,usExchange
0,20201130,USD,U.S. Dollar,1
1,20201130,CAD,Canadian Dollar,1.29755
2,20201130,EUR,Euro Dollar,0.836925
3,20201130,LEGACY,LEGACY,visionNA
4,20201130,ATS,Austrian Schilling,11.5163
5,20201130,BEF,Belgian Franc,33.7615
6,20201130,FIM,Finnish Markka,4.97613
7,20201130,FRF,French Franc,5.48987
8,20201130,DEM,German Deutschmark,1.63688
9,20201130,IEP,Irish Punt,0.659132


In [10]:
#-- Add daterange

vcode = '''
JS getArrayFrom: [ CurrencyMap factset masterList first: 5 ]
   for: "usExchange" over: (^date to: ^date - 3 monthEnds by: 1 monthEnds )
'''
result = vc.runvisionJSON(vcode)
pd.DataFrame(result['array'])

,date,id,usExchange
0,20201130,USD,1
1,20201031,USD,1
2,20200930,USD,1
3,20200831,USD,1
4,20201130,CAD,1.29755
5,20201031,CAD,1.33335
6,20200930,CAD,1.33575
7,20200831,CAD,1.30295
8,20201130,EUR,0.836925
9,20201031,EUR,0.85848


In [11]:
#-- Let's do something with holdings data now
#---  we can add any expression that navigate to data

vcode = '''
JS getArrayFrom: 
    [ Account findId: "sml" .
        holdings sortDown: [ percentOfPort ] . numberElements first: 3 
    ]
   for: "position, security id, account id, shares , percentOfPort, mapEntry factset price"
   over: (^date, ^date - 1 monthEnds, ^date - 1 yearEnds)
'''
result =  vc.runvisionJSON(vcode)
pd.DataFrame(result['array'])

,account id,date,id,mapEntry factset price,percentOfPort,position,security id,shares
0,SML,20191231,23726610-12483W10-12/31/2019,28.08,0.586450,2,DAR,163711000
1,SML,20191231,50187A10-12483W10-12/31/2019,137.76,0.520558,3,LHCG,29620340
2,SML,20191231,58468810-12483W10-12/31/2019,84.94,0.805522,1,MDCO,74337690
3,SML,20201031,85285720-12483W10-10/30/2020,223.24,0.577462,3,STMP,17576000
4,SML,20201031,64049M20-12483W10-10/30/2020,39.23,0.637589,1,NEO,110431000
5,SML,20201031,49714P10-12483W10-10/30/2020,187.47,0.585437,2,KNSL,21218620
6,SML,20201130,11434010-12483W10-11/27/2020,73.03,0.652694,2,BRKS,73760000
7,SML,20201130,G1890L10-12483W10-11/27/2020,36.00,0.655812,1,CPRI,150345000
8,SML,20201130,64049M20-12483W10-11/27/2020,47.36,0.633710,3,NEO,110431000


In [12]:
#--  We're in python now.  Fix the column order

pd.DataFrame(result['array'], columns = ["account id", "security id", "date", "percentOfPort"])

,account id,security id,date,percentOfPort
0,SML,DAR,20191231,0.586450
1,SML,LHCG,20191231,0.520558
2,SML,MDCO,20191231,0.805522
3,SML,STMP,20201031,0.577462
4,SML,NEO,20201031,0.637589
5,SML,KNSL,20201031,0.585437
6,SML,BRKS,20201130,0.652694
7,SML,CPRI,20201130,0.655812
8,SML,NEO,20201130,0.633710


In [13]:
#--  The runvisionJSON function assumes you are returning a JSON structure from Vision
#--  The getArrayFrom: for: over: method will automatically build and return this structure
#--  You will get a python error if the returned object from vision is not JSON
vcode = 'Currency masterListCodes'
#-- run next line to see the python error
#-- vc.runvisionJSON(vcode)


## Building Your Own JSON Structure in Vision

There are a number of new methods defined at the Vision class **JS** that enable you to set key-value pairs in a dictionary object and return the object to your client, in this case, your python program.  The values can be integers, doubles, strings, other objects, or arrays.  

#### Messages defined at **JS** 

* *newObject*              --  create an object that is rendered as an external dictionary object
* *newArrayFrom: list .*   --  converts a vision list to an external array object

* *returnJSON: obj .*   --  return the supplied obj to the client  (note to t&l - repackage this one a bit)

#### Messages define at **JS newObject**

* *set: key to val .*  --  sent to an external object to set supplied key to supplied val






In [14]:
#-- Sample Query: Create an object that contains information about an account

#--  Create the Vision Code to Run
query = '''

!acct <- Account findId: "SP50" ;

#-- Create a JS object with some key/value pairs
!data <- JS newObject
  set: "id" to: acct id .
  set: "name" to: acct name .
  set: "count" to: acct holdings count .
  set: "date" to: acct holdingsDate asInteger ;

#-- Return the object as JSON ( note: jsParm is equivalent of sending asSelf)
JS jsObject returnJSON: data jsParam

'''

#--  Run the vision code
#-- Call Vision to run the expression, returning a JS object
response = vc.runvisionJSON(query)

#--- Print the response and its type
print ("the response is of type ", type(response))
print (response)

#-- When we use the JSON call, the response is converted to a python dictionary
#-- so we can access elements directly
print ("We can access pieces of the response:")
print ("Id is: " , response['id'], " and name is: ", response['name'])

the response is of type  <class 'dict'>
{'id': 'SP50', 'name': 'S&P 500', 'count': 506, 'date': 20201127}
We can access pieces of the response:
Id is:  SP50  and name is:  S&P 500


In [16]:
#-- Expanding our example to return a named array containing the holdings

query = '''

!acct <- Account findId: "r.1000" ;

#--  Create a JS object with some key/value pairs
!data <- JS newObject 
   set: "id" to: acct id .
   set: "name" to: acct name .
   set: "count" to: acct holdings count .
   set: "date" to: acct holdingsDate asInteger ;
   
#-- create an array of JS holding objects
!holdObjects <-
acct holdings sortDown: [ percentOfPort ] . first: 5 . 
 send: [ GlobalWorkspace JS newObject 
         set: "date" to: ^date asString .
         set: "acctId" to: account code .
         set: "secId" to: security code .
         set: "weight" to: percentOfPort .
       ] ;
       
#-- add a key-value pair to data object that references this vision list as an array
data set: "holdings" to: (JS newArrayFrom: holdObjects ) ;   
   
#-- Return the object as JSON   
 JS jsObject returnJSON: data jsParam

'''

#--  Call Vision to run the expression
getHoldings = vc.runvisionJSON(query)

print (getHoldings)

{'id': 'R.1000', 'name': 'Russell 1000', 'count': 1012, 'date': 20201127, 'holdings': [{'date': '11/30/2020', 'acctId': 'R.1000', 'secId': '03783310', 'weight': 5.489112010975741}, {'date': '11/30/2020', 'acctId': 'R.1000', 'secId': '59491810', 'weight': 4.701944214840047}, {'date': '11/30/2020', 'acctId': 'R.1000', 'secId': '02313510', 'weight': 3.9526544152917}, {'date': '11/30/2020', 'acctId': 'R.1000', 'secId': '30303M10', 'weight': 1.9461207962229552}, {'date': '11/30/2020', 'acctId': 'R.1000', 'secId': '02079K30', 'weight': 1.5649558739686766}]}


In [17]:
#-- By default, we are printing the entire js object.  DataFrames help here

print("Holdings for: " + getHoldings['id'] + "  " + getHoldings['name'])
pd.DataFrame(getHoldings['holdings'])

Holdings for: R.1000  Russell 1000


,acctId,date,secId,weight
0,R.1000,11/30/2020,03783310,5.489112
1,R.1000,11/30/2020,59491810,4.701944
2,R.1000,11/30/2020,02313510,3.952654
3,R.1000,11/30/2020,30303M10,1.946121
4,R.1000,11/30/2020,02079K30,1.564956


## Packaged Applications with the Applay Class

The **Applay** class has been created to support building structured applications in Vision that return JSON dictionary objects.   Applications are defined at the class **Applay Lib** as methods that can read inputs from a supplied JSON dictionary object.

Within your method you can use the following messages:

* *getNewObject*  -- returns a js dictionary object that responds to the *set: key to: val* message
* *getNewArrayFrom: list*  -- returns an array from a list of js dictionary objects
* *responseObject*  --  the js dictionary object that will be returned to your calling program
* *updateStatusWithSuccess*  -- marks the status flag in the retuned js dictionary object
* *postErrorWithMessage: errorString* -- returns the error message supplied



In [18]:
#---  Create an Application called Tester
#---    params = acctId, date, num
app = '''

Applay Lib defineMethod: [ | Tester |

!id <-   ^self getParam: "acctId" ;
!date <- ^self getParam: "date" . asDate else: ^date ;
!max <-  ^self getParam: "num" . asInteger  ;
!entity <- GlobalWorkspace Account findId: id ;
entity isntDefault
ifTrue:
  [ !holdings <- date evaluate: [ entity holdings ] ;
    max isntNA
      ifTrue: [ :holdings <- holdings sortDown: [ percentOfPort ] . first: max ] ;
    ^self responseObject
        set: "id" to: entity id .
        set: "name" to: entity name .
       ;
     !constits <- holdings
        send: [ ^my getNewObject
                    set: "acctId" to: account code .
                    set: "date" to: date formatUsingShortName .
                    set: "secId"
                     to: security descriptiveData factsetId .
                    set: "name" to: security name .
                    set: "weight" to: percentOfPort .
                    set: "mval" to: totalMarketValue .
                ] ;
    ^self responseObject
         set: "constituents" to: (^self getNewArrayFrom: constits);
    ^self updateStatusWithSuccess ;
    ]
ifFalse:
   [ ^self postErrorWithMessage: ("Id " concat: id . concat: " not found.")
   ] ;

^self
] ;


'''
#-- Define the Method and confirm that it is there
print(vc.runvision(app))
print(vc.runvision('Applay Lib showMessages'))

#--  Run the Application
params = {'acctId' : "SP50", 'num' : 3}
response = vc.runapp ("Tester", params)
response['status']
print (response)

pd.DataFrame(response['response']['constituents'])

#--- Run it with Bad inputs
#print (vc.runapp("Tester", {'acctId' : "badId"}))


Messages Defined At Class: Applay Lib
AccountHoldings                 * New Message
AccountSecurityXTab             * New Message
AllAccountSummary               * New Message
BenchmarkMemberChanges          * New Message
SecurityAccountXTab             * New Message
SecurityDateXTab                * New Message
Tester                          * New Message
TopHoldingsByAccount            * New Message
WhoOwns                         * New Message
asSelf                          * New Message
classDescriptor                 * New Message
defaultInstance                 * New Message
isLib                           * New Message
whatAmI                         * New Message


{'appName': 'Tester', 'status': {'code': 0, 'message': 'Success'}, 'response': {'id': 'SP50', 'name': 'S&P 500', 'constituents': [{'acctId': '00000117', 'date': '27-Nov-2020', 'secId': '03783310', 'name': 'Apple Inc.', 'weight': 6.210302338598431, 'mval': 1874345591905.5999}, {'acctId': '00000117', 'date': '27-Nov

,acctId,date,mval,name,secId,weight
0,00000117,27-Nov-2020,1.874346e+12,Apple Inc.,03783310,6.210302
1,00000117,27-Nov-2020,1.628786e+12,Microsoft Corp,59491810,5.396685
2,00000117,27-Nov-2020,1.360437e+12,Amazon.com Inc,02313510,4.507559
